In [ ]:
import pandas as pd
df = pd.read_csv("lv-comments-2019_main.csv", sep='\t')

In [ ]:
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()
df['polarity']=label.fit_transform(df['polarity'])
df.head()

In [ ]:
dfx = pd.read_csv("lv-comments-2019_main.csv", sep='\t')
#calculate, assign new columns
dfx['Characters'] = df['content'].str.len()
dfx['Words'] = df['content'].str.split().str.len()
#export to CSV
df.to_csv('out.csv', index=False)
#print values
mean_characters = dfx['Characters'].mean()
mean_words = dfx['Words'].mean()
print(mean_characters)
print(mean_words)

In [ ]:
import seaborn as sns
sns.countplot('polarity',data=df)

In [ ]:
stopwordsfile = open('data/stopwords-lv.txt', mode='r', encoding='utf-8')
stopwords_lv = stopwordsfile.read()
stopwordsfile.close()
stopwords_lv = stopwords_lv[1:].split('\n')

!pip install LatvianStemmer
import LatvianStemmer

import gzip
import re
dictionary_file = gzip.open('data/lemmatization_dictionary_lv.dat.gz', 'rt', encoding='utf-8')
lemmatization_dictionary = {}
for line in dictionary_file:
    values = line.split('\t')
    lemmatization_dictionary[values[0]] = values[1][:-1]
dictionary_file.close()

def lemmatize_lv(word):
    if word in lemmatization_dictionary:
        return lemmatization_dictionary[word]
    else:
        return word
    
def process_message(review_text):
    new_review_text = re.sub(r'@[^ ]+', '', review_text)
    new_review_text = re.sub("http[^\s]+", '', new_review_text)
    new_review_text = re.sub(r'[^\w\s]', ' ', new_review_text)
    new_review_text = re.sub('_', ' ', new_review_text)
    new_review_text = re.sub('[0-9]+', '', new_review_text)
    words = new_review_text.lower().split()
    words = [w for w in words if not w in stopwords_lv]
    #lemmatizer
    words = [lemmatize_lv(word) for word in words]
    # stemmer
    #words = [LatvianStemmer.stem(word) for word in words]
    return (" ".join(words))

In [ ]:
df['content']=df['content'].apply(str)

In [ ]:
df['clean_content']=df['content'].apply(lambda x: process_message(x))

In [ ]:
df['clean_content']=df['content']

In [ ]:
df.clean_content.head()

In [ ]:
df.head()

In [ ]:
 pd.options.display.max_colwidth = 900  # set a value as your need

In [ ]:
df.shape

In [ ]:
testing_df.shape

In [ ]:
df.describe()

In [ ]:
import matplotlib.pyplot as plt 
from wordcloud import WordCloud,STOPWORDS
def createWrdCloudForSentiment(sentiment):
    sentiment_num = 1 if sentiment== 'Hate' else 0
    temp_df = df[df.polarity==sentiment_num]
    words = " ".join(temp_df.clean_content)
    cleaned_words = " ".join([w for w in words.split()
                                  if 'http' not in w
                                    and not w.startswith('@')
                                    and w!='RT'])

    wrdcld = WordCloud(stopwords=STOPWORDS,
                      background_color='black',
                      width=1500,
                      height=1000).generate(cleaned_words)
    plt.figure(figsize=(10,10))
    plt.imshow(wrdcld)
    plt.axis('off')
    plt.show

In [ ]:
createWrdCloudForSentiment('Hate')

In [ ]:
createWrdCloudForSentiment('Positive')

In [ ]:
from sklearn.model_selection import train_test_split
df, testing_df =  train_test_split(df, test_size = 0.2, random_state = 42)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
#vectorizer = TfidfVectorizer()
#train_tfidf_model = vectorizer.fit_transform(df.clean_content)
#test_tfidf_model = vectorizer.transform(testing_df.clean_content)

vectorizer = CountVectorizer()
train_tfidf_model = vectorizer.fit_transform(df.clean_content)
test_tfidf_model = vectorizer.transform(testing_df.clean_content)

#vectorizer = HashingVectorizer()
#train_tfidf_model = vectorizer.fit_transform(df.clean_content)
#test_tfidf_model = vectorizer.transform(testing_df.clean_content)





In [ ]:
train_tfidf_model

In [ ]:
train_tfidf = pd.DataFrame(train_tfidf_model)
train_tfidf

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier



cls = [LogisticRegression(),
       MultinomialNB(),
       BernoulliNB(),
       DecisionTreeClassifier(),
       RandomForestClassifier(n_estimators=200),
       KNeighborsClassifier(n_neighbors = 5),
       SVC(),
       LinearSVC(),
       XGBClassifier(),
       SGDClassifier(),
       AdaBoostClassifier(),
       MLPClassifier(),
       BaggingClassifier()]

cls_name = []

In [ ]:
test_tfidf_model

In [ ]:
df.polarity.count()

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

lbl_actual = testing_df.polarity
i = 0
accuracy = []
for cl in cls:
    model = cl.fit(train_tfidf_model,df.polarity)
    lbl_pred = model.predict(test_tfidf_model)
    a = (100*accuracy_score(lbl_pred, lbl_actual))
    a = round(a,2)
    accuracy.append(a)
    cls_name.append(cl.__class__.__name__)
    print ("{}  Accuracy Score : {}%".format(cls_name[i],a))
    print ( classification_report(lbl_pred, lbl_actual))
    i +=1

In [ ]:
plt.bar(cls_name, accuracy)
plt.xticks(rotation=70)

In [ ]:
# Predict accuracy
def getModelAccuracy_LogicalReg(model_name, sampled_train_df) :
    vectorizer = TfidfVectorizer()
    sampled_train_tfidf_model = vectorizer.fit_transform(sampled_train_df.clean_content)
    sampled_test_tfidf_model = vectorizer.transform(testing_df.clean_content)

    #Prediction
    sample_model = LogisticRegression().fit(sampled_train_tfidf_model,sampled_train_df.polarity)
    lg_lbl_pred = sample_model.predict(sampled_test_tfidf_model)
    a = (100*accuracy_score(lg_lbl_pred, lbl_actual))
    a = round(a,2)
    print ("{}  Accuracy Score : {}%".format(model_name,a))
    #print(type(a))
    return float(a)

In [ ]:
def getModelAccuracy_RFC(model_name, sampled_train_df) :

    
    vectorizer = TfidfVectorizer()
    sampled_train_tfidf_model = vectorizer.fit_transform(sampled_train_df.clean_content)
    sampled_test_tfidf_model = vectorizer.transform(testing_df.clean_content)

    sample_model = RandomForestClassifier(n_estimators=200).fit(sampled_train_tfidf_model,sampled_train_df.polarity)
    lg_lbl_pred = sample_model.predict(sampled_test_tfidf_model)
    a = (100*accuracy_score(lg_lbl_pred, lbl_actual))
    a = round(a,2)
    print ("{}  Accuracy Score : {}%".format(model_name,a))
    return float(a)

In [ ]:
log_accuracy = []
rfc_accuracy = []

In [ ]:
a = getModelAccuracy_LogicalReg("Train dataset", df)

log_accuracy.append(a)

In [ ]:
log_accuracy

In [ ]:
a = getModelAccuracy_RFC("Train dataset", df)
rfc_accuracy.append(a)

In [ ]:
rfc_accuracy

In [ ]:
def createPieChartFor(t_df):
    Lst = 100*t_df.value_counts()/len(t_df)
    
    # set data for pie chart
    labels = t_df.value_counts().index.values
    sizes =  Lst 
    
    # set labels
    fig1, ax1 = plt.subplots()
    ax1.pie(sizes, labels=labels, autopct='%1.2f%%', shadow=True, startangle=90)
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.show()



In [ ]:
createPieChartFor(df.polarity)

In [ ]:
!pip install Flask-JSON

In [ ]:
!pip install tweepy

In [ ]:
import requests
import os
import json
import pandas as pd
import csv
import datetime
import dateutil.parser
import unicodedata
import time

In [ ]:
import tweepy
MY_BEARER_TOKEN = "that is secret"
client = tweepy.Client(bearer_token=MY_BEARER_TOKEN)


In [ ]:
query = "gobzems lang:lv"
start_time = "2022-03-22T00:00:00Z"
end_time = "2022-03-28T12:00:00Z"

tweets = client.search_recent_tweets(query=query,
                                     start_time=start_time,
                                     end_time=end_time,
                                     tweet_fields = ["created_at", "text", "source"],
                                     user_fields = ["name", "username", "location", "verified", "description"],
                                     max_results = 100,
                                     expansions='author_id'
                                     )


In [ ]:
print(len(tweets.data))
print(len(tweets.includes["users"]))

In [ ]:
first_tweet = tweets.data[0]
dict(first_tweet)

In [ ]:
import pandas as pd
tweet_info_ls = []
for tweet, user in zip(tweets.data, tweets.includes['users']):
    tweet_info = {
        'created_at': tweet.created_at,
        'text': tweet.text,
        'source': tweet.source,
        'name': user.name,
        'username': user.username,
        'location': user.location,
        'verified': user.verified,
        'description': user.description
    }
    tweet_info_ls.append(tweet_info)
tweets_df = pd.DataFrame(tweet_info_ls)
tweets_df.head()

In [ ]:
tweets_df.to_csv('gobzems.csv')

In [ ]:
tweets_df.drop(columns=['created_at', 'source', "name", "username", "location", "verified", "description"])

In [ ]:
tweets_df['text']=tweets_df['text'].apply(str)

In [ ]:
tweets_df['clean_text']=tweets_df['text'].apply(lambda x: process_message(x))

In [ ]:
tweets_df.clean_text.head()

In [ ]:
sampled_train_tfidf_model = vectorizer.fit_transform(df.clean_content)
sampled_test_tfidf_model = vectorizer.transform(tweets_df.clean_text)

In [ ]:
sample_model = BaggingClassifier().fit(sampled_train_tfidf_model,df.polarity)
lg_lbl_pred = sample_model.predict(sampled_test_tfidf_model)

In [ ]:
lg_lbl_pred_df = pd.DataFrame({'tweet_cleaned' : tweets_df.clean_text,
                            'tweet' : tweets_df.text,
                            'polarity' : lg_lbl_pred})
lg_lbl_pred_df.head()

In [ ]:
lg_lbl_pred_df.polarity.value_counts()

In [ ]:
lg_lbl_pred_df.to_csv('gobzems-output.csv', index=False)